In [21]:
# 导入相关库
from xgboost import XGBRegressor  # 导入XGBoost回归器
from sklearn.ensemble import RandomForestRegressor  # 导入随机森林回归器
from sklearn.linear_model import LinearRegression  # 导入线性回归模型
from sklearn.model_selection import KFold, cross_val_score, train_test_split as TTS # 导入交叉验证、交叉验证评分及数据集划分
from sklearn.metrics import mean_squared_error  # 导入均方误差计算的方法

import pandas as pd  # 导入处理数据的pandas库
import numpy as np  # 导入数值计算库Numpy
import matplotlib.pyplot as plt  # 导入绘图库matplotlib
from time import time  # 导入时间模块中的time函数
import datetime  # 导入日期时间模块

In [37]:
#读取文件
file_path = "D:\data\heart+disease\processed.cleveland.data"
data = pd.read_csv(file_path, delimiter=',')
print(data)


     63.0  1.0  1.0.1  145.0  233.0  1.0.2  2.0  150.0  0.0  2.3  3.0 0.0.1  \
0    67.0  1.0    4.0  160.0  286.0    0.0  2.0  108.0  1.0  1.5  2.0   3.0   
1    67.0  1.0    4.0  120.0  229.0    0.0  2.0  129.0  1.0  2.6  2.0   2.0   
2    37.0  1.0    3.0  130.0  250.0    0.0  0.0  187.0  0.0  3.5  3.0   0.0   
3    41.0  0.0    2.0  130.0  204.0    0.0  2.0  172.0  0.0  1.4  1.0   0.0   
4    56.0  1.0    2.0  120.0  236.0    0.0  0.0  178.0  0.0  0.8  1.0   0.0   
..    ...  ...    ...    ...    ...    ...  ...    ...  ...  ...  ...   ...   
297  45.0  1.0    1.0  110.0  264.0    0.0  0.0  132.0  0.0  1.2  2.0   0.0   
298  68.0  1.0    4.0  144.0  193.0    1.0  0.0  141.0  0.0  3.4  2.0   2.0   
299  57.0  1.0    4.0  130.0  131.0    0.0  0.0  115.0  1.0  1.2  2.0   1.0   
300  57.0  0.0    2.0  130.0  236.0    0.0  2.0  174.0  0.0  0.0  2.0   1.0   
301  38.0  1.0    3.0  138.0  175.0    0.0  0.0  173.0  0.0  0.0  1.0     ?   

     6.0  0  
0    3.0  2  
1    7.0  1  
2    3.0 

In [38]:
# 替换'MissingValue'为NaN
data.replace('?', np.nan, inplace=True)
# 转换数据类型
data = data.astype(float)

In [39]:
# 设置特征和标签
X = data.iloc[:, :13]  # 前13列特征
y = data.iloc[:, 13]   # 第14列标签

In [40]:
# 打印特征
print("特征:")
print(X)

特征:
     63.0  1.0  1.0.1  145.0  233.0  1.0.2  2.0  150.0  0.0  2.3  3.0  0.0.1  \
0    67.0  1.0    4.0  160.0  286.0    0.0  2.0  108.0  1.0  1.5  2.0    3.0   
1    67.0  1.0    4.0  120.0  229.0    0.0  2.0  129.0  1.0  2.6  2.0    2.0   
2    37.0  1.0    3.0  130.0  250.0    0.0  0.0  187.0  0.0  3.5  3.0    0.0   
3    41.0  0.0    2.0  130.0  204.0    0.0  2.0  172.0  0.0  1.4  1.0    0.0   
4    56.0  1.0    2.0  120.0  236.0    0.0  0.0  178.0  0.0  0.8  1.0    0.0   
..    ...  ...    ...    ...    ...    ...  ...    ...  ...  ...  ...    ...   
297  45.0  1.0    1.0  110.0  264.0    0.0  0.0  132.0  0.0  1.2  2.0    0.0   
298  68.0  1.0    4.0  144.0  193.0    1.0  0.0  141.0  0.0  3.4  2.0    2.0   
299  57.0  1.0    4.0  130.0  131.0    0.0  0.0  115.0  1.0  1.2  2.0    1.0   
300  57.0  0.0    2.0  130.0  236.0    0.0  2.0  174.0  0.0  0.0  2.0    1.0   
301  38.0  1.0    3.0  138.0  175.0    0.0  0.0  173.0  0.0  0.0  1.0    NaN   

     6.0  
0    3.0  
1    7.0  
2 

In [41]:
# 打印标签
print("标签:")
print(y)


标签:
0      2.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
297    1.0
298    2.0
299    3.0
300    1.0
301    0.0
Name: 0, Length: 302, dtype: float64


In [119]:
import xgboost as xgb

#随机分割训练数据集和测试数据集
X_train, X_test, y_train, y_test = TTS(X, y, test_size=0.4, random_state=42)

# 转换为DMatrix，XGBoost的专用数据结构
dtrain = xgb.DMatrix(X_train, label=y_train, missing=np.nan)
dtest = xgb.DMatrix(X_test, label=y_test, missing=np.nan)

In [43]:
print("y_train:", y_train)

y_train: 286    0.0
281    3.0
197    0.0
158    0.0
164    0.0
      ... 
188    2.0
71     1.0
106    1.0
270    0.0
102    0.0
Name: 0, Length: 226, dtype: float64


In [120]:
param = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 5 ,
    'eta': 0.25,
    'max_depth': 3,
    'min_child_weight': 1,
    'subsample': 0.3,
    'colsample_bytree': 0.4,
    'gamma': 0.3,
    'lambda': 2,
    'alpha': 2,
    'eval_metric': 'logloss',
    'seed': 42
}

num_round = 75
bst = xgb.train(param, dtrain, num_round)

from sklearn.metrics import accuracy_score
# 预测
preds = bst.predict(dtest)

# 计算准确率
accuracy = accuracy_score(y_test, preds)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 59.50%
